In [1]:
import gym_examples
import numpy as np
import gym
env = gym.make('gym_examples/GridWorld-v3', render_mode="human", size=3, rewards=np.array([[1,2,3],[4,5,6],[7,8,9]]), costs=np.array([[1,2,3],[4,5,6],[7,8,9]]))

In [2]:
observation, info = env.reset()

for _ in range(10):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward_cost, terminated, truncated, info = env.step(action)
    print(observation, reward_cost, terminated, truncated, info)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

/Users/noahliniger/opt/miniconda3/envs/FORL/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/noahliniger/opt/miniconda3/envs/FORL/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:252: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(


{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([1, 0])} [4, 4] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 0])} [1, 1] False False {}
{'agent': array([0, 1])} [2, 2] False False {}
{'agent': array([0, 1])} [2, 2] False False {}


In [5]:
import numpy as np
np_random.integers(0, self.size, size=2, dtype=int)

NameError: name 'np_random' is not defined

In [12]:
A = np.array([[1,2,3]])

In [13]:
A[0,0]

1

In [19]:
print(A.value())

AttributeError: 'numpy.ndarray' object has no attribute 'value'